## **Pré-Processamento Auxiliar pro Docker - Tem que otimizar a estrutura - Combinar com VFSSImageDataset**

In [3]:
import sys
sys.path.append('../../')

In [4]:
import torch
from src.vfss_dataset import VFSSImageDataset
from src.data_extraction.video_frame import create_video_frame_df, load_video_frame_metadata_from_csv
from src.split_data import split_data_k_fold

from src.utils import plot_image_with_mask
from torchvision import transforms as T

from tqdm import tqdm
from torch.utils.data import DataLoader, ConcatDataset
import numpy as np
import pandas as pd
import kornia
import cv2 as cv

from notebooks.playground_VC.unet.unet_regression import UNet
from notebooks.playground_VC.posenet.posenet import PoseNet
from torch import optim
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

In [5]:
PROJECT_NAME = "vfss-data-split"

def get_script_directory() -> str:
    '''Get the directory of the current script.'''
    script_path = os.path.abspath(__file__)
    script_dir = os.path.dirname(script_path)
    return script_dir

def get_project_root_directory() -> str:
    '''Get the root directory of the project (assumed to be the parent of the script directory).'''
    script_dir = get_script_directory()
    
    if not PROJECT_NAME in script_dir:
        raise RuntimeError(f"Cannot determine project root directory. '{PROJECT_NAME}' not found in script path.")

    project_root = script_dir.split(PROJECT_NAME)[0]
    project_root = os.path.join(project_root, PROJECT_NAME)
    return project_root

def save_frame_from_video_path(path_video: str, path_output: str, frame_index: int):
    ''' Load a specific frame from a video file and save it to a specified path. '''

    root_dir = get_project_root_directory()
    relative_path = os.path.join(root_dir, path_video)

    if not os.path.exists(relative_path):
        raise FileNotFoundError(f"File not found: {relative_path}")

    cap = cv.VideoCapture(relative_path)

    if not cap.isOpened():
        raise IOError(f"Cannot open video file: {relative_path}")
    
    total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    
    if frame_index <= 0 or frame_index > total_frames:
        cap.release()
        raise IndexError(f"Frame index {frame_index} out of range for video with {total_frames} frames.")

    # Set frame position and read it
    cap.set(cv.CAP_PROP_POS_FRAMES, frame_index - 1)
    ok, frame = cap.read()
    cap.release()

    if not ok:
        raise IOError(f"Error reading frame {frame_index} from video file: {relative_path}")

    # Convert BGR to RGB
    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    image = Image.fromarray(frame)
    
    # Create output directory if it doesn't exist
    output_dir = os.path.dirname(path_output)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
    
    # Save the image
    image.save(path_output)
    print(f"Frame {frame_index} saved to: {path_output}")
    
    return image

def collect_video_frames(df_video_frames: pd.DataFrame, path_video: str, path_output: str):
    for i in range(len(df_video_frames)):
        df_aux = df_video_frames.loc[i]
        frame = df_aux["frame_ID"]
        video_frame = df_aux["video_frame"]
        filename_output = os.path.join(path_output, video_frame + ".png")
        save_frame_from_video_path(path_video, filename_output, frame)


In [ ]:
videos_dir = 'C:\\Users\\vinic\\OneDrive\\Documentos\\Faculdade\\Doutorado\\Pesquisa_INCA\\Disfagia_Vinicius\\data\\videos\\' 
labels_dir = 'C:\\Users\\vinic\\OneDrive\\Documentos\\Faculdade\\Doutorado\\Pesquisa_INCA\\vfss-data-split\\data\\rotulos\\anotacoes-tecgraf\\'
frames_dir = 'data\\video_frame\\'

# Setta o ambiente e carrega o conjunto de dados
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

video_frame_df = create_video_frame_df(
    videos_dir=videos_dir,
    labels_dir=labels_dir,
    target='points'
)

collect_video_frames(df_video_frames, videos_dir, frames_dir)

SyntaxError: invalid syntax (940692702.py, line 13)